In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from pathlib import Path 
from fastai.tabular import *
from fastai.metrics import error_rate

In [3]:
bs = 32
np.random.seed(42)

In [4]:
path = %pwd
path = Path(f'{path}/../csv/')

In [5]:
df = pd.read_csv(path/'raw.csv')

In [6]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,31992,31993,31994,31995,31996,31997,31998,31999,filename,classes
0,-0.001923,-0.001488,-0.001092,0.000636,-0.000439,0.002301,0.000686,0.000867,0.006124,0.008916,...,-0.002832,-0.003827,0.002071,0.001415,-0.001815,-0.003134,-0.003944,-0.004587,blink_29,blink
1,0.061929,0.061929,0.061929,0.061929,0.061929,0.061929,0.061929,0.061929,0.061929,0.061929,...,0.061929,0.061929,0.061929,0.061929,0.061929,0.061929,0.061929,0.061929,blink_7,blink
2,0.016476,0.006415,0.010163,0.013385,0.012466,0.016891,0.017876,0.011519,0.017348,0.013792,...,-0.003748,0.002971,-0.003928,-0.005492,-0.001918,-0.001472,0.008276,-0.004121,blink_8,blink
3,0.024835,0.024835,0.024835,0.024835,0.024835,0.024835,0.024835,0.024835,0.024835,0.024835,...,0.024835,0.024835,0.024835,0.024835,0.024835,0.024835,0.024835,0.024835,blink_28,blink
4,-0.022613,-0.022613,-0.022613,-0.022613,-0.022613,-0.022613,-0.022613,-0.022613,-0.022613,-0.022613,...,-0.022613,-0.022613,-0.022613,-0.022613,-0.022613,-0.022613,-0.022613,-0.022613,blink_11,blink
